# Ассистент 1 - LM на основе n-грамм

Перед вами - первое дополнительное задание повышенной сложности, в рамках которого вам предстоит начать разработку телеграм-бота с генеративными моделями.

Цель данного ноутбука - помочь влиться в разработку ассистента. В данном ноутбуке написан код для "обучения" LM на основе n-грамм, для генерации с помощью нее текста, а также сохранение и загрузка модели и токенизатора.

Относитесь к данному заданию максимально творчески - любую часть кода можно менять под ваши нужды и желания, можно оптимизировать, добавлять методы генерации, использовать любые данные, обучать сколь угодно "большую" модель. 

При этом вам стоит быть готовыми со всеми техническими проблеми справляться самому - именно так обычно происходит в реальной жизни в реальных проектах :) 

Поэтому отдельно подчеркну:
* если что-то сломалось после ваших изменений - подразумевается, что вы сами найдете проблему и исправите
* если вы ничего не трогали, но что-то не работает у нас - подразумевается, что вы сами найдете проблему и исправите :) 

Главный критерий выполнености данного задания - телеграм-бот, генерирующий текст и использующий обозначенный в задании подход (в случае данного ноутбука - n-граммная модель в любой ее реализации).

_

Для обучения качественной модели вам потребуются датасеты. В ноутбуке составлен маленький игрушечный датасет, вам для улучшения качества потребуется данные в большем количестве и более качественные, а также другие параметры модели и генерации (например, размер контекста побольше). 

С нормальным датасетом и правильными параметрами даже такой простой моделью можно добиться адекватного качества генерации текста (возможно не очень человечный, но вполне связный текст).

Датасеты можно найти и выбрать тут (желательно на русском, вам так будет понятней качество и в целом полезней):
https://huggingface.co/datasets
  
Можете найти наиболее интересный для себя датасет (можете сделать модель как смешной, так и полезной), либо выбрать любой из этих датасетов
* https://huggingface.co/datasets/Den4ikAI/russian_dialogues
* https://huggingface.co/datasets/Georgii/russianPoetry
* https://huggingface.co/datasets/IgorVolochay/russian_jokes

In [1]:
import re
import pickle 
from itertools import chain
from datetime import datetime
from collections import defaultdict

from typing import List, Dict, Optional, Iterable, Tuple

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import tokenizers
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.processors import TemplateProcessing

Токенизатор разбивает текст на слова. Можно попробовать другие способы токенизации

In [2]:
class Tokenizer:
    def __init__(self,
                 token_pattern: str = '\w+|[\!\?\,\.\-\:]',
                 eos_token: str = '<EOS>',
                 pad_token: str = '<PAD>',
                 unk_token: str = '<UNK>'):
        self.token_pattern = token_pattern
        self.eos_token = eos_token
        self.pad_token = pad_token
        self.unk_token = unk_token
        
        self.special_tokens = [self.eos_token, self.pad_token, self.unk_token]
        self.vocab = None
        self.inverse_vocab = None
    
    def text_preprocess(self, input_text: str) -> str:
        """ Предобрабатываем один текст """
        # input_text = ... # приведение к нижнему регистру
        input_text = input_text.lower()
        input_text = re.sub('\s+', ' ', input_text) # унифицируем пробелы
        input_text = input_text.strip()
        return input_text
    
    def build_vocab(self, corpus: List[str]) -> None:
        assert len(corpus)
        all_tokens = set()
        for text in corpus:
            all_tokens |= set(self._tokenize(text, append_eos_token=False))
        self.vocab = {elem: ind for ind, elem in enumerate(all_tokens)}
        special_tokens = [self.eos_token, self.unk_token, self.pad_token]
        for token in special_tokens:
            self.vocab[token] = len(self.vocab)
        self.inverse_vocab = {ind: elem for elem, ind in self.vocab.items()}
        return self
        
    def _tokenize(self, text: str, append_eos_token: bool = True) -> List[str]:
        text = self.text_preprocess(text)
        tokens = re.findall(self.token_pattern, text)
        if append_eos_token:
            tokens.append(self.eos_token)
        return tokens
    
    def encode(self, text: str, append_eos_token: bool = True) -> List[str]:
        """ Токенизируем текст """
        tokens = self._tokenize(text, append_eos_token)
        ids = [self.vocab.get(token, self.vocab[self.unk_token]) for token in tokens]
        return ids
    
    def decode(self, input_ids: Iterable[int], remove_special_tokens: bool = False) -> str:
        assert len(input_ids)
        assert max(input_ids) < len(self.vocab) and min(input_ids) >= 0
        tokens = []
        for ind in input_ids:
            token = self.inverse_vocab[ind]
            if remove_special_tokens and token in self.special_tokens:
                continue
            tokens.append(token)
        text = ' '.join( tokens )
        return text
    
    def save(self, path: str) -> bool:
        data = {
            'token_pattern': self.token_pattern,
            'eos_token': self.eos_token,
            'pad_token': self.pad_token,
            'unk_token': self.unk_token,
            'special_tokens': self.special_tokens,
            'vocab': self.vocab,
            'inverse_vocab': self.inverse_vocab,
        }
        
        with open(path, 'wb') as fout:
            pickle.dump(data, fout)
            
        return True
        
    def load(self, path: str) -> bool:
        with open(path, 'rb') as fin:
            data = pickle.load(fin)
            
        self.token_pattern = data['token_pattern']
        self.eos_token = data['eos_token']
        self.pad_token = data['pad_token']
        self.unk_token = data['unk_token']
        self.special_tokens = data['special_tokens']
        self.vocab = data['vocab']
        self.inverse_vocab = data['inverse_vocab']

Класс для задания параметров генерации, так удобней писать логику для валидации параметров и разные другие доп методы

In [3]:
class GenerationConfig:
    def __init__(self, **kwargs):
        """
        Тут можно задать любые параметры и их значения по умолчанию
        Значения для стратегии декодирования decoding_strategy: ['max', 'top-p']
        """
        self.temperature = kwargs.pop("temperature", 1.0)
        self.max_tokens = kwargs.pop("max_tokens", 32)
        self.sample_top_p = kwargs.pop("sample_top_p", 0.9)
        self.decoding_strategy = kwargs.pop("decoding_strategy", 'max')
        self.remove_special_tokens = kwargs.pop("remove_special_tokens", False)
        self.validate()
        
    def validate(self):
        """ Здесь можно валидировать параметры """
        if not (1.0 > self.sample_top_p > 0):
            raise ValueError('sample_top_p')
        if self.decoding_strategy not in ['max', 'top-p']:
            raise ValueError('decoding_strategy')

Сама LM на основе n-грамм. Тут используется сглаживание Лапласа (можно поменять на метод backoff при желании), а также есть ряд параметров, сильно влияющий на качество генерации. Один из параметров генерации - стратегия генерации. 

Когда мы получили вероятности для следующего токена, мы по этим вероятностям хотим выбрать этот следующий токен.

Можно просто семплировать из этого распределения - но тогда есть шанс, что будут очень маловероятные токены.

Можно брать самый вероятный токен - но это плохо повлияет на разнообразие и "человечность" языка

Можно воспользовать подходом top-p - семплировать только из тех токенов, которые наиболее вероятны (их вероятности суммируются в заданный p)

Можно проверить, что top-p будет генерировать более интересный текст чем max

Также обратите внимание на параметр температуры. В случае top-p и семплирования, чем больше делаешь температуру, тем меньше отличаются друг от друга вероятности (распределение стремится к равномерному, даже если исходное распределение имело вполне себе выраженные максимумы), и текст становится более случайным (и разнообразным)

In [4]:
class StatLM:
    def __init__(self, 
                 tokenizer: Tokenizer,
                 context_size: int = 2,
                 alpha: float = 0.1
                ):
        
        assert context_size >= 2
        
        self.context_size = context_size
        self.tokenizer = tokenizer
        self.alpha = alpha
        
        self.n_gramms_stat = defaultdict(int)
        self.nx_gramms_stat = defaultdict(int)
        
    def get_token_by_ind(ind: int) -> str:
        return self.tokenizer.vocab.get(ind)
    
    def get_ind_by_token(token: str) -> int:
        return self.tokenizer.inverse_vocab.get(token, self.tokenizer.inverse_vocab[self.unk_token])
        
    def train(self, train_texts: List[str]):
        for sentence in tqdm(train_texts, desc='train lines'):
            sentence_ind = self.tokenizer.encode(sentence)
            for i in range(len(sentence_ind) - self.context_size):
                
                seq = tuple(sentence_ind[i: i + self.context_size - 1])
                self.n_gramms_stat[seq] += 1
                
                seq_x = tuple(sentence_ind[i: i + self.context_size])
                self.nx_gramms_stat[seq_x] += 1
                
            seq = tuple(sentence_ind[len(sentence_ind) - self.context_size:])
            self.n_gramms_stat[seq] += 1
            
    def sample_token(self, 
                     token_distribution: np.ndarray,
                     generation_config: GenerationConfig) -> int:
        if generation_config.decoding_strategy == 'max':
            return token_distribution.argmax()
        elif generation_config.decoding_strategy == 'top-p':
            token_distribution = sorted(list(zip(token_distribution, np.arange(len(token_distribution)))),
                                        reverse=True)
            total_proba = 0.0
            tokens_to_sample = []
            tokens_probas = []
            for token_proba, ind in token_distribution:
                tokens_to_sample.append(ind)
                tokens_probas.append(token_proba)
                total_proba += token_proba
                if total_proba >= generation_config.sample_top_p:
                    break
            # для простоты отнормируем вероятности, чтобы суммировались в единицу
            tokens_probas = np.array(tokens_probas) / generation_config.temperature
            tokens_probas = tokens_probas / tokens_probas.sum()
            return np.random.choice(tokens_to_sample, p=tokens_probas)
        else:
            raise ValueError(f'Unknown decoding strategy: {generation_config.decoding_strategy}')
            
    def save_stat(self, path: str) -> bool:
        stat = {
            'n_gramms_stat': self.n_gramms_stat,
            'nx_gramms_stat': self.nx_gramms_stat,
            'context_size': self.context_size,
            'alpha': self.alpha
        }
        with open(path, 'wb') as fout:
            pickle.dump(stat, fout)
            
        return True
    
    def load_stat(self, path: str) -> bool:
        with open(path, 'rb') as fin:
            stat = pickle.load(fin)
            
        self.n_gramms_stat = stat['n_gramms_stat']
        self.nx_gramms_stat = stat['nx_gramms_stat']
        self.context_size = stat['context_size']
        self.alpha = stat['alpha']
            
        return True
        
    def get_stat(self) -> Dict[str, Dict]:
        
        n_token_stat, nx_token_stat = {}, {}
        for token_inds, count in self.n_gramms_stat.items():
            n_token_stat[self.tokenizer.decode(token_inds)] = count
        
        for token_inds, count in self.nx_gramms_stat.items():
            nx_token_stat[self.tokenizer.decode(token_inds)] = count
        
        return {
            'n gramms stat': self.n_gramms_stat,
            'n+1 gramms stat': self.nx_gramms_stat,
            'n tokens stat': n_token_stat,
            'n+1 tokens stat': nx_token_stat,
        }
    
    def _get_next_token(self, 
                        tokens: List[int],
                        generation_config: GenerationConfig) -> (int, str):
        denominator = self.n_gramms_stat.get(tuple(tokens), 0) + self.alpha * len(self.tokenizer.vocab)
        numerators = []
        for ind in self.tokenizer.inverse_vocab:
            numerators.append(self.nx_gramms_stat.get(tuple(tokens + [ind]), 0) + self.alpha)
        
        token_distribution = np.array(numerators) / denominator
        max_proba_ind = self.sample_token(token_distribution, generation_config)
        
        next_token = self.tokenizer.inverse_vocab[max_proba_ind]
        
        return max_proba_ind, next_token
            
    def generate_token(self, 
                       text: str, 
                       generation_config: GenerationConfig
                      ) -> Dict:
        tokens = self.tokenizer.encode(text, append_eos_token=False)
        tokens = tokens[-self.context_size + 1:]
        
        max_proba_ind, next_token = self._get_next_token(tokens, generation_config)
        
        return {
            'next_token': next_token,
            'next_token_num': max_proba_ind,
        }
    
    
    def generate_text(self, text: str, 
                      generation_config: GenerationConfig
                     ) -> Dict:
        
        all_tokens = self.tokenizer.encode(text, append_eos_token=False)
        tokens = all_tokens[-self.context_size + 1:]
        
        next_token = None
        while next_token != self.tokenizer.eos_token and len(all_tokens) < generation_config.max_tokens:
            max_proba_ind, next_token = self._get_next_token(tokens, generation_config)
            all_tokens.append(max_proba_ind)
            tokens = all_tokens[-self.context_size + 1:]
        
        new_text = self.tokenizer.decode(all_tokens, generation_config.remove_special_tokens)
        
        finish_reason = 'max tokens'
        if all_tokens[-1] == self.tokenizer.vocab[self.tokenizer.eos_token]:
            finish_reason = 'end of text'
        
        return {
            'all_tokens': all_tokens,
            'total_text': new_text,
            'finish_reason': finish_reason
        }
    
    def generate(self, text: str, generation_config: Dict) -> str:
        return self.generate_text(text, generation_config)['total_text']

Эта функция напрямую используется в телеграм боте для получения модели и конфига генерации

In [5]:
def construct_model():
    config = {
        'temperature': 1.0,
        'max_tokens': 32,
        'sample_top_p': 0.9,
        'decoding_strategy': 'top-p',
    }

    stat_lm_path = 'models/stat_lm/stat_lm.pkl'
    tokenizer_path = 'models/stat_lm/tokenizer.pkl'
    
    tokenizer = Tokenizer()
    tokenizer.load(tokenizer_path)
        
    stat_lm = StatLM(tokenizer)
    stat_lm.load_stat(stat_lm_path)

    generation_config = GenerationConfig(temperature=config['temperature'],
                                         max_tokens=config['max_tokens'],
                                         sample_top_p=config['sample_top_p'],
                                         decoding_strategy=config['decoding_strategy'],
                                         remove_special_tokens=True)

    kwargs = {'generation_config': generation_config}
    return stat_lm, kwargs

### Обучаем на игрушечных данных

Для демонстрации того, что происходит, возьмем несколько коротких цитат Джейсона Стэтхема отсюда:

https://dzen.ru/a/ZRFaGN_gKhX6xTWW

In [6]:
generation_config = GenerationConfig(temperature = 1.0, max_tokens = 32,
                                     sample_top_p = 0.9, decoding_strategy = 'max',
                                     remove_special_tokens=True)

In [7]:
texts = [
    'Взял нож - режь, взял дошик - ешь.',
    'Никогда не сдавайтесь, идите к своей цели! А если будет сложно – сдавайтесь.',
    'Запомни: всего одна ошибка – и ты ошибся.',
    'В жизни всегда есть две дороги: одна — первая, а другая — вторая.',
    'Делай, как надо. Как не надо, не делай.',
    'Работа не волк. Никто не волк. Только волк волк.',
    'Работа не волк. Работа - ворк. А волк - это ходить.',
    'Работа',
    ]

train_texts = texts[:-1]
test_text = texts[-1]

In [8]:
tokenizer = Tokenizer().build_vocab(train_texts)

In [9]:
dict(list(tokenizer.vocab.items())[:5])

{'нож': 0, 'цели': 1, 'сложно': 2, 'никто': 3, 'ты': 4}

In [10]:
# класс, который позволяем строить и использовать языковую модель на основе n-грамм
stat_lm = StatLM(tokenizer, context_size=2, alpha=0.1) # , sample_top_p = None

# "обучаем" модель - считаем статистики
stat_lm.train(train_texts)

train lines:   0%|          | 0/7 [00:00<?, ?it/s]

In [11]:
print(stat_lm.generate("вот такие пироги!", generation_config))

! а волк . никто не волк . никто не волк . никто не волк . никто не волк . никто не волк . никто не волк . никто


In [12]:
tokenizer.save('models/stat_lm/tokenizer.pkl')
stat_lm.save_stat('models/stat_lm/stat_lm.pkl')

True

Тут мы для токенизатора сохраняем только спецтокены и словарь, для модели - параметры и статистики n-грамм и n+1-грамм. Потом в телеграм боте подгружаем именно эти параметры

Когда обучите модель на большом датасете, советую посмотреть на распределение вероятностей для следующего слова при разных входах

### смотрим как конструировать

In [16]:
model, kwargs = construct_model()

model.generate("дошик", **kwargs)

'дошик - режь одна . взял нож другая как вторая две дороги как ворк . как надо , а другая вторая режь сдавайтесь надо дороги делай взял дошик в режь всегда'